In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
sns.set_style('darkgrid')
pd.set_option('display.max_columns', None)

In [3]:
df = pd.read_csv('Network_anomaly_data.csv')
df.head()

,duration,protocoltype,service,flag,srcbytes,dstbytes,land,wrongfragment,urgent,hot,numfailedlogins,loggedin,numcompromised,rootshell,suattempted,numroot,numfilecreations,numshells,numaccessfiles,numoutboundcmds,ishostlogin,isguestlogin,count,srvcount,serrorrate,srvserrorrate,rerrorrate,srvrerrorrate,samesrvrate,diffsrvrate,srvdiffhostrate,dsthostcount,dsthostsrvcount,dsthostsamesrvrate,dsthostdiffsrvrate,dsthostsamesrcportrate,dsthostsrvdiffhostrate,dsthostserrorrate,dsthostsrvserrorrate,dsthostrerrorrate,dsthostsrvrerrorrate,attack,lastflag
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.00,0.00,0.00,150,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal,20
1,0,udp,other,SF,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,1,0.0,0.0,0.0,0.0,0.08,0.15,0.00,255,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal,15
2,0,tcp,private,S0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,123,6,1.0,1.0,0.0,0.0,0.05,0.07,0.00,255,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,19
3,0,tcp,http,SF,232,8153,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.2,0.2,0.0,0.0,1.00,0.00,0.00,30,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal,21
4,0,tcp,http,SF,199,420,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,30,32,0.0,0.0,0.0,0.0,1.00,0.00,0.09,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal,21


In [4]:
df.select_dtypes(include='object')

,protocoltype,service,flag,attack
0,tcp,ftp_data,SF,normal
1,udp,other,SF,normal
2,tcp,private,S0,neptune
3,tcp,http,SF,normal
4,tcp,http,SF,normal
...,...,...,...,...
125968,tcp,private,S0,neptune
125969,udp,private,SF,normal
125970,tcp,smtp,SF,normal
125971,tcp,klogin,S0,neptune


In [5]:
df['protocoltype'].value_counts()

,count
protocoltype,
tcp,102689
udp,14993
icmp,8291


In [32]:
pd.pivot_table(data=df[df['attack'] != 'normal'],index='protocoltype',columns='attack',values='duration',aggfunc='count',fill_value=0,margins=True)

attack,back,buffer_overflow,ftp_write,guess_passwd,imap,ipsweep,land,loadmodule,multihop,neptune,nmap,perl,phf,pod,portsweep,rootkit,satan,smurf,spy,teardrop,warezclient,warezmaster,All
protocoltype,,,,,,,,,,,,,,,,,,,,,,,
icmp,0,0,0,0,0,3117,0,0,0,0,981,0,0,201,5,0,32,2646,0,0,0,0,6982
tcp,956,30,8,53,11,482,18,9,7,41214,265,3,4,0,2926,7,2184,0,2,0,890,20,49089
udp,0,0,0,0,0,0,0,0,0,0,247,0,0,0,0,3,1417,0,0,892,0,0,2559
All,956,30,8,53,11,3599,18,9,7,41214,1493,3,4,201,2931,10,3633,2646,2,892,890,20,58630


In [6]:
df['service'].value_counts()

,count
service,
http,40338
private,21853
domain_u,9043
smtp,7313
ftp_data,6860
...,...
tftp_u,3
http_8001,2
aol,2


In [7]:
tmp = df.copy()
tmp['is_attack'] = tmp['attack'].apply(lambda x: 'normal' if x=='normal' else 'attack')
tmp['is_attack'].value_counts()

,count
is_attack,
normal,67343
attack,58630


In [8]:
tmp['flag'].unique()

array(['SF', 'S0', 'REJ', 'RSTR', 'SH', 'RSTO', 'S1', 'RSTOS0', 'S3',
       'S2', 'OTH'], dtype=object)

In [9]:
tmp[tmp['protocoltype'] != 'tcp']['flag'].value_counts()

,count
flag,
SF,23284


In [10]:
tmp[tmp['protocoltype'] == 'tcp']['flag'].value_counts()

,count
flag,
SF,51661
S0,34851
REJ,11233
RSTR,2421
RSTO,1562
S1,365
SH,271
S2,127
RSTOS0,103


In [20]:
flag_df = pd.pivot_table(data=tmp,index='flag',columns='is_attack',values='duration',aggfunc='count',fill_value=0,margins=True).reset_index()
flag_df['attack%'] = (flag_df['attack']/flag_df['All'])*100
flag_df['normal%'] = (flag_df['normal']/flag_df['All'])*100
flag_df

is_attack,flag,attack,normal,All,attack%,normal%
0,OTH,35,11,46,76.086957,23.913043
1,REJ,8540,2693,11233,76.025995,23.974005
2,RSTO,1343,219,1562,85.979513,14.020487
3,RSTOS0,103,0,103,100.000000,0.000000
4,RSTR,2275,146,2421,93.969434,6.030566
5,S0,34497,354,34851,98.984247,1.015753
6,S1,4,361,365,1.095890,98.904110
7,S2,8,119,127,6.299213,93.700787
8,S3,4,45,49,8.163265,91.836735
9,SF,11552,63393,74945,15.413970,84.586030


In [27]:
flag_df[flag_df['normal%'] > flag_df['attack%']]

is_attack,flag,attack,normal,All,attack%,normal%
6,S1,4,361,365,1.095890,98.904110
7,S2,8,119,127,6.299213,93.700787
8,S3,4,45,49,8.163265,91.836735
9,SF,11552,63393,74945,15.413970,84.586030
11,All,58630,67343,125973,46.541719,53.458281


In [26]:
attack_flags = ['OTH','REJ','RSTO','RSTOS0','RSTR','S0','SH']
pd.pivot_table(data=tmp[(tmp['flag'].isin(attack_flags)) & (tmp['is_attack'] != 'normal')],index='flag',columns='attack',values='duration',aggfunc='count',fill_value=0,margins=True)

attack,back,buffer_overflow,guess_passwd,imap,ipsweep,land,neptune,nmap,portsweep,satan,warezclient,All
flag,,,,,,,,,,,,
OTH,0,0,0,0,0,0,0,0,35,0,0,35
REJ,0,0,0,0,396,0,5671,0,509,1964,0,8540
RSTO,0,1,45,0,30,0,1216,0,44,6,1,1343
RSTOS0,0,0,0,0,0,0,0,0,103,0,0,103
RSTR,90,0,4,0,0,0,0,0,2179,1,1,2275
S0,0,0,0,0,0,18,34326,0,54,99,0,34497
SH,0,0,0,4,0,0,0,265,0,0,0,269
All,90,1,49,4,426,18,41213,265,2924,2070,2,47062


In [36]:
df['attack'].value_counts()

,count
attack,
normal,67343
neptune,41214
satan,3633
ipsweep,3599
portsweep,2931
smurf,2646
nmap,1493
back,956
teardrop,892


In [35]:
df['attack'].nunique()

23

In [47]:
df[df['attack'] == 'pod']['protocoltype'].value_counts()

,count
protocoltype,
icmp,201


In [54]:
flag_attack = pd.pivot_table(data=df,index='flag',columns='attack',values='duration',aggfunc='count',fill_value=0)

In [55]:
from scipy.stats import chi2_contingency



chi_stat, p_value, dof, exp_freq = chi2_contingency(flag_attack) # chi_stat, p_value, df, expected value

print("p_value:",p_value)
alpha = 0.05

if p_value < alpha:
    print("Reject H0 Attack and Flag are not independent")
else:
    print("Fail to reject H0 Attack and Flag are independent")

p_value: 0.0
Reject H0 Attack and Flag are not independent


In [11]:
pd.pivot_table(data=tmp,index='land',columns='is_attack',values='duration',aggfunc='count')

is_attack,attack,normal
land,,
0,58612,67336
1,18,7


In [12]:
df[df['land'] == 1]['attack'].value_counts()

,count
attack,
land,18
normal,7


In [13]:
df[df['attack'] == 'land'].shape

(18, 43)